In [1]:
import pandas as pd
import numpy as np
import re  # string mani
import os  # read fold info

#### Notes:
* wave use: ADNI1, ADNIGO, ADNI2

* CSF values in UPENNBIOMK_MASTER & ADNIMERGE is different, couldn't find how ADNIMERGE use it
    -> multiple batches in UPENNBIOMK_MASTER, use the median -> standardized to ADNI1
    
* find values start from MCI -> check 6m, 12m, 24m follow up

* make the MCI with A/T/N values as a specific subgroup -> compare the prediction & feature importance

Methods applied: catboost, light gbm, xgboost, gbm, random forest, logistic regression

evaluation: MCC, accuracy...

In [11]:
# functions:
def d_overview(df):
    print(f"Dimension: {df.shape}")
    output = df.apply(lambda x: (x.dtype,x.isna().mean(),x.unique()), axis=0).T
    output.columns = ["dtype", "NaN_precentage","unique"]
    return output.sort_values(by="NaN_precentage", ascending=False)
    
def keep_median_row(group):
    if 'MEDIAN' in group['BATCH'].values:
        return group[group['BATCH'] == 'MEDIAN']
    else:
        return group.head(1)
def basic_clean(df, printna=False):
#     if printna:
#         print(df.apply(lambda x:sum(x.isin(["-1","-4",-1,-4]))))
    df.replace([-1,-4,"-1","-4"], np.nan, inplace=True)
    if 'update_stamp' in df.columns:
        df.drop('update_stamp', axis=1, inplace=True)
    df = df.loc[:, (df.isna().mean() <= 0.5) &   # rm >50% missingness
                ~(df.columns.str.contains('DATE'))  # drop dates
               ].reset_index(drop=True) 
    df['RID'] = df['RID'].astype(int)
    return df
    

In [12]:
path = "../../ADNI_testing/"
biomarker = pd.read_csv(f"{path}UPENNBIOMK_MASTER.csv")     # adni1
diagnosis = pd.read_csv(f"{path}DXSUM_PDXCONV_ADNIALL.csv")
symptoms = pd.read_csv(f"{path}ADSXLIST.csv")
bl_symptoms = pd.read_csv(f"{path}BLSCHECK.csv")
adnimerge = pd.read_csv("../../ADNIMERGE.csv")

In [8]:
## biomarker
# biomarker.apply(lambda x:sum(x.isin(["-1","-4",-1,-4])))  # no -1,-4 in biomarker
# keep the median value or single row batch:
biomarker = biomarker.groupby(['RID', 'VISCODE'], as_index=False) \
.apply(keep_median_row).reset_index(drop=True)
biomarker = biomarker.loc[:,['RID','VISCODE','ABETA','TAU','PTAU']] # keep useful cols
# diagnosis=diagnosis.loc[diagnosis["Phase"]!="ADNI3"] # rm ADNI3 for diagnosis
# diagnosis = basic_clean(diagnosis)
symptoms = basic_clean(symptoms)
bl_symptoms = basic_clean(bl_symptoms)
## merge the 4 dataset:
fulldata = pd.merge(biomarker, symptoms, left_on=['RID', 'VISCODE'],
                 right_on=['RID', 'VISCODE2'], how="left", suffixes=["","_y"])
fulldata = pd.merge(
    fulldata,bl_symptoms, on=['RID', 'VISCODE2'], how="left")
fulldata=fulldata.loc[:,~(fulldata.columns.str.contains(r"(_x|_y)"))]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [22]:
#adnimerge.replace([-1,-4,"-1","-4"], np.nan, inplace=True)

#x = d_overview(adnimerge.loc[adnimerge["DX"]=="MCI"])
x.loc[x["NaN_precentage"]<=0.3]

,dtype,NaN_precentage,unique
Hippocampus,float64,0.295877,"[6869.0, 6439.0, 6451.0, 6373.0, 6213.0, 5348...."
Ventricles,float64,0.234236,"[39605.0, 39726.0, 38527.0, 40768.0, 45254.0, ..."
FDG_bl,float64,0.227769,"[nan, 1.07846, 1.3896, 0.93668, 1.06972, 1.157..."
WholeBrain,float64,0.215643,"[1154980.0, 1116280.0, 1117390.0, 1095210.0, 1..."
LDELTOTAL,float64,0.205538,"[4.0, nan, 5.0, 3.0, 0.0, 6.0, 1.0, 7.0, 2.0, ..."
FLDSTRENG_bl,object,0.199879,"[1.5 Tesla MRI, 3 Tesla MRI, nan]"
ICV,float64,0.192401,"[1679440.0, 1661130.0, 1656740.0, 1648510.0, 1..."
FSVERSION,object,0.192199,[Cross-Sectional FreeSurfer (FreeSurfer Versio...
IMAGEUID,float64,0.192199,"[64631.0, 64636.0, 59357.0, 79178.0, 132252.0,..."
Entorhinal_bl,float64,0.1538,"[3983.0, 2277.0, nan, 4381.0, 2784.0, 3105.0, ..."


In [13]:
pd.set_option('display.max_rows', 500) 
display(d_overview(adnimerge))

Dimension: (16222, 116)


,dtype,NaN_precentage,unique
PIB_bl,float64,0.990507,"[nan, 1.49, 2.22, 1.195, 1.8925, 1.18, 1.6725,..."
PIB,float64,0.986253,"[nan, 2.3575, 2.1275, 2.0725, 2.19, 2.27, 2.35..."
FBB,float64,0.968315,"[nan, 1.0714, 1.0372, 0.9902, 0.9959, 0.9602, ..."
FBB_bl,float64,0.935396,"[nan, 1.0714, 1.0372, 0.9902, 0.9959, 0.9602, ..."
PTAU,object,0.853964,"[nan, 22.83, 24.18, 13.29, 13.3, 33.43, 34.04,..."
TAU,object,0.853902,"[nan, 239.7, 251.7, 153.1, 159.7, 337.0, 334.1..."
ABETA,object,0.853902,"[nan, 741.5, 601.4, 1501.0, 1176.0, 547.3, 472..."
AV45,float64,0.810134,"[nan, 1.3496, 0.9804, 0.9851, 0.994, 0.9416, 0..."
FDG,float64,0.772346,"[1.33615, 1.1086, 1.07796, 1.11626, 1.06134, n..."
DIGITSCOR,float64,0.76575,"[34.0, 25.0, 19.0, 21.0, 16.0, 24.0, 30.0, 28...."


In [122]:
# diagnosis["DXCHANGE"].value_counts()
pd.crosstab(fulldata["DX"], fulldata["ORIGPROT"])

ORIGPROT,ADNI1,ADNI2,ADNIGO
DX,,,
CN,328,344,5
Dementia,311,178,4
MCI,410,425,166


In [165]:
# adnimerge
# adnimerge.columns
fulldata = pd.merge(adnimerge, biomarker, on=['RID', 'VISCODE'], how="right")
# df = df.dropna(thresh=2)
# adnimerge[["PTAU", "TAU", "ABETA"]].dropna(thresh=2)
fulldata.loc[:,["PTAU_y","PTAU_x", "TAU_y", "TAU_x"]]


y = (fulldata.PTAU_y - fulldata.PTAU_y.mean())/fulldata.PTAU_y.std()
x = fulldata["PTAU_x"][-fulldata["PTAU_x"].str.contains("[><]", na=False)]
x = x.astype(float)
x = (x - x.mean())/x.std()

0      -0.449641
1      -0.515232
2      -0.092248
3      -0.068014
4       0.751946
          ...   
2169    1.328331
2170   -0.122146
2171    1.907525
2172    0.220818
2173    0.109637
Length: 2174, dtype: float64

In [166]:
d_overview(fulldata)

Dimension: (2174, 119)


,dtype,NaN_precentage,unique
FBB_bl,float64,1.0,[nan]
FBB,float64,1.0,[nan]
PIB_bl,float64,0.99126,"[nan, 1.49, 1.195, 1.8925, 1.5675, 1.835, 2.23..."
PIB,float64,0.964121,"[nan, 2.0725, 2.19, 1.22, 2.1825, 1.83, 1.7525..."
DIGITSCOR,float64,0.568537,"[25.0, 21.0, 30.0, 38.0, 44.0, 49.0, 41.0, 27...."
DIGITSCOR_bl,float64,0.519779,"[25.0, 38.0, 49.0, 27.0, 30.0, 46.0, 60.0, 48...."
EcogSPOrgan_bl,float64,0.50966,"[nan, 3.4, 1.0, 1.16667, 1.33333, 2.0, 1.66667..."
EcogSPDivatt_bl,float64,0.49908,"[nan, 3.0, 1.25, 1.75, 2.5, 1.5, 2.75, 4.0, 2...."
EcogPtOrgan_bl,float64,0.4977,"[nan, 1.16667, 2.0, 1.5, 2.16667, 1.0, 3.8, 3...."
EcogSPVisspat_bl,float64,0.49494,"[nan, 1.28571, 1.57143, 1.0, 1.14286, 1.42857,..."


In [173]:



pd.crosstab(fulldata.DX_bl, fulldata.VISCODE=="m12")

VISCODE,False,True
DX_bl,,
AD,277,74
CN,491,96
EMCI,397,0
LMCI,585,156
SMC,95,0
